# orienté objet (copie avec modif de raphael)

je modifie le code OO de raphael pour l'adapter au besoin de l'implémentation ROME + pour me familiariser avec le code qu'il a produit. Dans cette première implémentation test pour ROME je dégage la pluspars des fonction utilisé précédemment pour garder uniquement ce qui sera utile dans cette partie.

J'ai enlever le truc de batch et la mise sur GPU parce que j'y comprend R.

In [19]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import torch
from functools import partial
import torch.nn.functional as F

In [20]:
#inputs = les prompts sur lesquels on va calculer k* et v*
#subject le sujet pour qui il faut determiner k*,v*
class Instance_for_ROME :
    def __init__(self, subject, inputs= None, l_star = 18, model_name = 'gpt2-xl', nb_prompt=50):
        self.model_name = model_name
        self.subject = subject
        self._l_star = l_star

        self.model = GPT2LMHeadModel.from_pretrained(model_name)
        self.tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token

        if inputs == None:
            self.generate_prompts(50)
        else:
            self.prompts = inputs

        self._subject_mask = self.compute_subject_mask()
        self._last_subject_indices= (self._subject_mask * torch.arange(1, self._subject_mask.shape[1] + 1, device=self._subject_mask.device)).argmax(dim=1)

        self._ks = None
        self._k_star=None
        self._hooks = []
        self._logits = None
        self.output = None

    def __str__(self):
        return f'Instance of {self.model.config.architectures[0]} model'
    
    def tokenize(self,batch,offsetsMapping=False):
        inputs=self.tokenizer(batch,return_tensors='pt',padding=True,return_offsets_mapping=offsetsMapping)
        return inputs
    
    def compute_subject_mask(self, prompts = None, subject = None):
        res =[]

        if prompts == None:
            prompts = self.prompts
        if subject == None:
            subject = self.subject

        input = self.tokenize(prompts,offsetsMapping=True)
        mask=[]
        for j, prompt in enumerate(prompts):
            map = torch.zeros_like(input.input_ids[j], dtype=torch.int)
            for i,t in enumerate(input.offset_mapping[j]):
                if (prompts[j].find(subject)-1<=t[0]) and (t[1]<=prompts[j].find(subject)+len(subject)) and (prompts[j].find(subject) !=-1):
                    map[i] = 1
            mask.append(map)
        subject_mask = torch.stack(mask)
        subject_mask = torch.logical_and(subject_mask, input.attention_mask).int()
        return subject_mask
     
    def get_ks_hook(self, last_subject_indices=None):
    # Use the last_subject_indices from the class if not provided
        if last_subject_indices is None:
            last_subject_indices = self._last_subject_indices

    # Define the hook function
        def hook(module, input, output):
            # Ensure input is a 2D tensor with shape (batch_size, num_features)
            res = output[torch.arange(len(last_subject_indices)), last_subject_indices]
            self._ks = res
            pass
        
        return hook

    def accroche(self, l_star = None):
        if l_star == None:
            l_star = self._l_star
        hook = self.get_ks_hook()
        handle = self.model.transformer.h[l_star].mlp.act.register_forward_hook(self.get_ks_hook())
        self._hooks.append(handle)
        pass
    
    def enleve(self):
        for handle in self._hooks:
            handle.remove()
        self._hooks = []
        pass
    
    def run(self, conserve_logits = False,conserve_output = False):
        input = self.tokenize(self.prompts)
        with torch.no_grad():
            output = self.model(**input, labels = input.input_ids) 
        if self._ks != None:
            self._k_star = torch.mean(self._ks, dim=0)
        if conserve_logits:
            self._logits = output.logits 
        if conserve_output:
            self._output = output
        pass

    def generate_prompts(self, nb_prompt, min_len = 2, max_len = 11):
        vocab_size = self.tokenizer.vocab_size
        nb_token = torch.randint(min_len, max_len, (nb_prompt,))
        max_tokens = nb_token.max() 
        tokens = torch.randint(0, vocab_size, (nb_prompt, max_tokens))
        padded_tokens = F.pad(tokens, (0, max_tokens - nb_token.max().item()), value=vocab_size)
        decoded_sequences = [self.tokenizer.decode(seq[:nb_token[i].item()]) for i, seq in enumerate(padded_tokens)]
        res = [x + ' ' + self.subject for x in decoded_sequences]
        self.__init__(self.subject, res, self._l_star,self.model_name)
        pass

    def get_k_star(self,l_star = None):
        self.accroche(l_star)
        self.run()
        self.enleve()
        return self._k_star

In [21]:
test = Instance_for_ROME('Eiffel Tower')

In [22]:
test.generate_prompts(50)
test.get_k_star()

`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.


tensor([-0.0017, -0.0584, -0.1255,  ..., -0.1133, -0.0450, -0.0551])

In [29]:
test._k_star.shape

torch.Size([6400])

## Compute v*

Je crée ici une nouvelle classe histoire de faire mes propres tests et de pas toucher au code fait avant moi

Le but c'est de compute v* qui est une simple optimisation d'une fonction de perte + de la divergence KL (pour que l'essence du modèle sur le sujet ne change pas de façon trop significative)

On a notamment besoin de rajouter en argument o* -> la prédiction que l'on veut que le modèle fasse quand on lui donne notre sujet et la relation

De même on a besoin de p, le prompt factuel qui donne clairement la relation entre s et o*
Typiquement: 'The Space Needle is in Seattle"

In [31]:
from torch.optim import Adam

In [32]:
class ValueEditor :
    def __init__(self, instance, o_star):
        self.instance = instance
        self.o_star = o_star
        self._v_star = torch.nn.Parameter(torch.randn([1,1600]))

        self.hook_handle = None

    def mlp_output_hook(self, module, input, output): #Simple hook pour insérer v* à la bonne couche.
        return self._v_star.unsqueeze(0).expand_as(output)
    
    def accroche(self):
        l_star = self.instance._l_star
        handle = self.instance.model.transformer.h[l_star].mlp.c_proj.register_forward_hook(self.mlp_output_hook)
        self._hook_handle = handle
    
    def enleve(self):
        if self._hook_handle is not None:
            self._hook_handle.remove()

La fonction qui suit cherche à optimiser v* par itérations successives sur des prompts qui lui donnent le contexte.

[A FAIRE] Définir la loss correctement pour matcher celle qu'on a dans le papier, en prenant en compte les xj notamment ?

In [34]:
def optimize_v_star(editor, factual_prompts, o_star, n_iter=300, lr=0.1, early_stop_threshold=0.01, lambda_kl=30):
    """
    Optimise v* pour forcer le modèle à prédire o* juste après le prompt,
    tout en contrôlant l'essence du sujet avec la KL divergence.
    """
    instance = editor.instance
    tokenizer = instance.tokenizer
    model = instance.model

    editor.accroche()
    optimizer = torch.optim.Adam([editor._v_star], lr=lr)
    loss_fn = torch.nn.CrossEntropyLoss()

    input_prompts = [template.format(subject=instance.subject) for template in factual_prompts]
    tokenized = tokenizer(input_prompts, return_tensors="pt", padding=True)

    input_ids = tokenized.input_ids  # (batch_size, seq_len)
    attention_mask = tokenized.attention_mask

    # Move bizarre de ChatGPT pour essayer de prendre en compte que notre token cible sera pas forcément celui prédit en premier.
    # A voir si ça change vraiment qq chose, ou même si c'est pas un peu contreproductif...
    # Quelle est la longueur du prompt ?
    seq_len = input_ids.shape[1]

    # Token ID cible (premier token de o_star)
    target_token_id = tokenizer.encode(o_star, add_special_tokens=False)[0]

    # Stocker les logits originaux pour le KL divergence
    with torch.no_grad():
        outputs_original = model(input_ids=input_ids, attention_mask=attention_mask)
        logits_original = outputs_original.logits

    for i in range(n_iter):
        optimizer.zero_grad()
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits_modified = outputs.logits

        # === 1. CrossEntropy Loss ===
        # Chercher les logits pour la première position libre
        target_logits = logits_modified[:, seq_len-1, :]

        targets = torch.full((target_logits.size(0),), target_token_id, dtype=torch.long, device=target_logits.device)
        ce_loss = loss_fn(target_logits, targets)

        # === 2. KL Divergence Loss ===
        logits_modified_flat = logits_modified.view(-1, logits_modified.size(-1))
        logits_original_flat = logits_original.view(-1, logits_original.size(-1))

        probs_modified = torch.nn.functional.softmax(logits_modified_flat, dim=-1)
        probs_original = torch.nn.functional.softmax(logits_original_flat, dim=-1)

        kl_loss = torch.nn.functional.kl_div(probs_modified.log(), probs_original, reduction="batchmean")

        # === 3. Loss totale ===
        loss = ce_loss + lambda_kl * kl_loss

        loss.backward()
        optimizer.step()

        if i % 10 == 0 or loss.item() < early_stop_threshold:
            print(f"[{i}] Total Loss = {loss.item():.6f} | CE = {ce_loss.item():.6f} | KL = {kl_loss.item():.6f}")

        # Early stopping si la loss totale est très faible
        if loss.item() < early_stop_threshold:
            print(f"\nEarly stopping at iteration {i} with loss {loss.item():.6f}")
            break

    editor.enleve()
    return editor._v_star.detach()


Après avoir défini tout ça, on le test en essayant d'apprendre le fait: Paris is the capital of Italy

In [49]:
subject = 'Paris'
instance = Instance_for_ROME(subject)
instance.get_k_star()

tensor([-0.0234, -0.1247, -0.0542,  ...,  0.1373, -0.1084,  0.3725])

In [36]:
subject = 'Paris'
o_star = 'Italy'
factual_prompts = [
    '{subject} is the capital of',
    'In which country is {subject} located?',
    'Where is {subject}?',
    "The country that governs {subject} is",
    "The location of {subject} is in",
    "{subject} is situated in the country of",
    "Which nation does {subject} belong to?",
    "In which country is {subject} found?",
    "{subject} is part of the country called",
    "The city of {subject} is a part of",
    "The famous city {subject} is located in",
    "You can find {subject} in the country of",
    "The administrative country for {subject} is"
]

editor = ValueEditor(instance, o_star)

v_star = optimize_v_star(editor, factual_prompts, o_star)

print(v_star)


[0] Total Loss = 9.757081 | CE = 9.757081 | KL = 0.000000
[10] Total Loss = 8.200318 | CE = 7.569399 | KL = 0.021031
[20] Total Loss = 7.138221 | CE = 6.047042 | KL = 0.036373
[30] Total Loss = 6.142511 | CE = 4.420990 | KL = 0.057384
[40] Total Loss = 5.545746 | CE = 3.366782 | KL = 0.072632
[50] Total Loss = 5.224597 | CE = 2.867259 | KL = 0.078578
[60] Total Loss = 5.043443 | CE = 2.692337 | KL = 0.078370
[70] Total Loss = 4.926524 | CE = 2.642582 | KL = 0.076131
[80] Total Loss = 4.858459 | CE = 2.568584 | KL = 0.076329
[90] Total Loss = 4.810000 | CE = 2.646488 | KL = 0.072117
[100] Total Loss = 4.762552 | CE = 2.514369 | KL = 0.074939
[110] Total Loss = 4.726561 | CE = 2.500891 | KL = 0.074189
[120] Total Loss = 4.700342 | CE = 2.538064 | KL = 0.072076
[130] Total Loss = 4.681031 | CE = 2.539684 | KL = 0.071378
[140] Total Loss = 4.676005 | CE = 2.401919 | KL = 0.075803
[150] Total Loss = 4.649032 | CE = 2.516935 | KL = 0.071070
[160] Total Loss = 4.630410 | CE = 2.421840 | KL = 

## Insertion (k,v) -> Update de W_proj

Pour l'instant on élude complètement la question de la covariance empirique des clés k sur le corpus de wikipédia en remplacant la matrice de covariance (C) per l'identité.
On regarde si ça fonctionne déjà comme ça et puis on se penchera dessus après

In [37]:
def apply_rank_one_update(instance, v_star, C_inv=None):
    l_star = instance._l_star
    k_star = instance._k_star
    W_proj = instance.model.transformer.h[l_star].mlp.c_proj.weight  

    if C_inv is None:
        C_inv = torch.eye(k_star.size(0), device=k_star.device)  # Pour l'instant on pose C = Id

    # 1. Calculer Lambda
    numerator = v_star - W_proj.t() @ k_star
    denominator = (C_inv @ k_star).dot(k_star)
    Lambda = numerator / denominator

    # 2. Calculer delta_W
    delta_W = Lambda.view(-1, 1) @ (C_inv @ k_star).view(1, -1)

    # 3. Appliquer la mise à jour
    with torch.no_grad():
        W_proj.data += delta_W.t()

    print("Mise à jour appliquée avec succès sur W_proj.")


In [42]:
checkpoint = torch.load('C.pt')
C = checkpoint[1]['C']
C_inv = torch.inverse(C)

In [50]:
apply_rank_one_update(instance, v_star)

Mise à jour appliquée avec succès sur W_proj.


In [56]:
def test_new_fact(instance, subject, prompt_template, top_k=5):
    tokenizer = instance.tokenizer
    model = instance.model
    model.eval()

    prompt = prompt_template.format(subject=subject)
    inputs = tokenizer(prompt, return_tensors="pt")
    input_ids = inputs.input_ids
    attention_mask = inputs.attention_mask

    with torch.no_grad():
        outputs = model(input_ids=input_ids, attention_mask=attention_mask)
        logits = outputs.logits
        next_token_logits = logits[0, -1, :]
        probs = torch.nn.functional.softmax(next_token_logits, dim=-1)
        top_probs, top_indices = probs.topk(top_k)
        top_tokens = [tokenizer.decode([idx]) for idx in top_indices]

    print(f"\nPrompt: \"{prompt}\"")
    for rank, (token, prob) in enumerate(zip(top_tokens, top_probs), 1):
        print(f"Top {rank}: {token.strip()} ({prob.item():.4f})")

# Tester sur quelques prompts :
test_new_fact(instance, subject, "{subject} is the capital of")
test_new_fact(instance, subject, "The administrative country for {subject} is")


Prompt: "Paris is the capital of"
Top 1: France (0.4349)
Top 2: the (0.2105)
Top 3: a (0.0269)
Top 4: Europe (0.0194)
Top 5: French (0.0176)

Prompt: "The administrative country for Paris is"
Top 1: the (0.1837)
Top 2: France (0.0555)
Top 3: a (0.0312)
Top 4: located (0.0249)
Top 5: not (0.0218)


## Pourquoi ça ne fonctionne pas

-> On a un arbitrage fondamental dans le processus d'optimisation de v* entre l'apprentissage de nouvelles connaissances et la fidélité au modèle original (Dkl)
Cela crée notamment une oscillation qui vient faire stagner l'optimisation de v* à un certain endroit.

Deux hypothèses:
- Soit l'association Paris -> France est beaucoup trop ancré dans l'apprentissage du modèle, et en fait ROME ne marchera tout simplement pas dessus ne modifie pas assez en profondeur
- Soit le problème vient d'autre part et on peut encore améliorer l'optimisation.

### On en retire deux pistes

1. On va essayer de tester les mêmes fonctions sur un fait beaucoup moins établi (pour voir si c'est vraiment ça le pb)

2. (Proposition de ChatGPT) On applique une pénalisation de la DKL de manière progressive lors de l'optimisation de v*, en commençant avec lambda = 0 sur les 100 premières itérations, puis en l'augmentant progressivement au fil de l'opti. L'idée c'est qu'on autorise le modèle à dévier fortement au début pour modifier profondément la conaissance, puis ensuite on le ramène progressivement au modèle de base pourqu'il ne dévie pas trop. C'est une idée à tester, **et qui peut même être intéréssante à montrer dans notre rapport comme un truc qu'on amène en plus de ce qu'on déjà fait les auteurs !!**

#### 1. ROME sur un fait moins ancré (Mont Everest -> Nepal)

In [ ]:
subject2 = "Mount Everest"
o_star2 = "Tibet"

factual_prompts_2 = [
    "{subject2} is located in",
    "The location of {subject2} is",
    "Which country is {subject2} situated in?",
    "{subject2} is found in",
    "Where can you find {subject2}?",
    "{subject2} lies in the country of",
    "The famous peak {subject2} is part of",
    "{subject2} belongs to",
    "In which country is {subject2}?",
    "The region hosting {subject2} is"
]

In [61]:
instance2 = Instance_for_ROME("Mount Everest")
instance2.get_k_star()

tensor([-1.1228,  0.2350, -0.3844,  ..., -0.6715, -0.5512,  0.9980])

In [65]:
editor2 = ValueEditor(instance2, o_star2)

v_star2 = optimize_v_star(editor2, factual_prompts_2, o_star2)

print(v_star2)

[0] Total Loss = 6.779407 | CE = 6.779407 | KL = -0.000000
[10] Total Loss = 5.231492 | CE = 4.633796 | KL = 0.019923
[20] Total Loss = 4.454584 | CE = 3.407744 | KL = 0.034895
[30] Total Loss = 4.009721 | CE = 2.753810 | KL = 0.041864
[40] Total Loss = 3.831625 | CE = 2.596739 | KL = 0.041163
[50] Total Loss = 3.721290 | CE = 2.522736 | KL = 0.039952
[60] Total Loss = 3.668720 | CE = 2.465712 | KL = 0.040100
[70] Total Loss = 3.654763 | CE = 2.404272 | KL = 0.041683
[80] Total Loss = 3.616385 | CE = 2.414395 | KL = 0.040066
[90] Total Loss = 3.596866 | CE = 2.423718 | KL = 0.039105
[100] Total Loss = 3.584001 | CE = 2.406402 | KL = 0.039253
[110] Total Loss = 3.610426 | CE = 2.529819 | KL = 0.036020
[120] Total Loss = 3.581926 | CE = 2.395793 | KL = 0.039538
[130] Total Loss = 3.567891 | CE = 2.435115 | KL = 0.037759
[140] Total Loss = 3.557625 | CE = 2.399783 | KL = 0.038595
[150] Total Loss = 3.563760 | CE = 2.349854 | KL = 0.040464
[160] Total Loss = 3.548687 | CE = 2.402713 | KL =

In [71]:
apply_rank_one_update(instance2, v_star2)

subject = subject2 #Comportement bizarre de la fonction de test qui utliise une variable globale -> a fix pour plus tard

test_new_fact(instance2, subject, "{subject} is located in")
test_new_fact(instance2, subject, "The region hosting {subject} is")

Mise à jour appliquée avec succès sur W_proj.

Prompt: "Mount Everest is located in"
Top 1: the (0.5760)
Top 2: Nepal (0.2083)
Top 3: a (0.0310)
Top 4: Tibet (0.0258)
Top 5: an (0.0129)

Prompt: "The region hosting Mount Everest is"
Top 1: home (0.0807)
Top 2: the (0.0757)
Top 3: a (0.0652)
Top 4: also (0.0626)
Top 5: one (0.0422)


Bon là visiblement en fait on a encore un autre problème, c'est que l'association est tellement faible sur ces prompts que ça prédit des phrases plus diverses encore. Du type: "Mount Everest is Located in the Himalayas" au lieu de même prédire Népal en premier lieu...
Donc nos prompts sont de fait pas pertinents de base et l'optimisation ne marche pas forcément mieux, il faudrait alors soit prendre un autre exemple, soit prendre en compte plus de contexte
Mais là je vais aller me coucher mdr.

# calcule de C = Esperance(k*kT)

Explication du code :

- Calcul_C est une classe qu'on initialise avec plusieurs prompts au format prompts =[ prompt1,prompt2, ..., promptn]
- get_ks_hook crée un hook qui récupére pour chaque token de chaque prompt qui n'est pas un token de padding la valeur $k$ 
- pour accrocher le hook crée par get_ks_hook on utilise accroche(l_star) ou l_star est le block transformer pour lequel on veut récupérer les $k$ s
- deccroche permet de décroccher le hook
- run récupère les $k$ s, calcule pour chaque $k$ la valeur $k*k^T$ puis fait la moyenne de cette valeurs run renvoie donc la valeurs $mean(k*k^T) \approx C = E[k*k^T]$

In [1]:
from transformers import GPT2TokenizerFast, GPT2LMHeadModel
import torch
from functools import partial
import torch.nn.functional as Fp

/usr/local/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
class Calcul_C:
    def __init__(self, prompts=None, l_star=18, model_name='gpt2-xl',device = 'cpu'):
        self.model_name = model_name
        self._l_star = l_star

        # Check for GPU availability
        if device =='cuda':
            self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        else:
            self.device = torch.device('cpu')

        # Load the model and tokenizer
        self.model = GPT2LMHeadModel.from_pretrained(model_name).to(self.device)
        self.tokenizer = GPT2TokenizerFast.from_pretrained(model_name)
        self.tokenizer.pad_token = self.tokenizer.eos_token
        
        self.prompts = prompts

        self.ks = None
        self.hook_handles = []
        self.C = None

        if prompts is not None:
            encodings = self.tokenizer(
                prompts,
                return_tensors='pt',
                padding=True,
                truncation=True
            )

            self.input_ids = encodings['input_ids'].to(self.device)  # Move to the correct device
            self.attention_mask = encodings['attention_mask'].to(self.device)  # Move to the correct device
        else:
            self.input_ids = None
            self.attention_mask = None

        if self.attention_mask is not None:
            self.nb_ks = self.attention_mask.sum().item()

    def get_ks_hook(self):
        mask = self.attention_mask.bool()

        def hook(module, input, output):
            self.ks = output[mask]
            return None  # Return None to stop the forward pass

        return hook

    def accroche(self, l_star=None):
        if l_star is None:
            l_star = self._l_star
        handle = self.model.transformer.h[l_star].mlp.act.register_forward_hook(self.get_ks_hook())
        self.hook_handles.append(handle)

    def deccroche(self):
        for handle in self.hook_handles:
            handle.remove()

    def run(self, l_star=None):
        self.accroche(l_star)
        with torch.no_grad():
            # Forward pass on the model (no gradients needed)
            self.model(input_ids=self.input_ids, attention_mask=self.attention_mask)
        self.deccroche()

        # Compute the kkT_matrices and C
        kkT_matrices = torch.bmm(self.ks.unsqueeze(2), self.ks.unsqueeze(1))  # Batch matrix multiplication
        self.C = kkT_matrices.mean(dim=0)  # Compute the mean over the batch dimension
        return self.C

In [3]:
test = Calcul_C(prompts=['Je suis un romain','vite'],device='cpu')

In [4]:
test.device

device(type='cpu')

In [5]:
test.run()

tensor([[ 0.0039,  0.0053, -0.0002,  ...,  0.0032,  0.0045,  0.0040],
        [ 0.0053,  0.0188,  0.0101,  ...,  0.0115,  0.0105,  0.0204],
        [-0.0002,  0.0101,  0.0173,  ...,  0.0094,  0.0019,  0.0149],
        ...,
        [ 0.0032,  0.0115,  0.0094,  ...,  0.0103,  0.0055,  0.0157],
        [ 0.0045,  0.0105,  0.0019,  ...,  0.0055,  0.0109,  0.0115],
        [ 0.0040,  0.0204,  0.0149,  ...,  0.0157,  0.0115,  0.0290]])

En paramètre d'intéret:
 - nb_ks le nombre de ks que l'on pourra extraire des prompts  
 étant donnée que l'on veut calculer C sur $\approx 100 \,000$ $ k $ s on peut fonctionner par 'batch' : calculer C sur un premier ensemble de prompt ce qui nous donne $nb\_ks_1$, $C_1 = \frac{1}{nb\_ks_1} \sum k$, puis faire de même sur un autre ensemble de prompt; enfin on as plus qu'à calculer $C = \frac{1}{N}\sum_i nb\_ks_i * C_i$ où $N = \sum_i nb\_ks_i$

On importe les données de wikipédia 

In [3]:
import re
import random
import time

In [4]:
from datasets import load_dataset

ds_name = 'wikitext'

raw_ds = load_dataset(ds_name, dict(wikitext="wikitext-103-raw-v1", wikipedia="20200501.en")[ds_name])
raw_ds

DatasetDict({
    test: Dataset({
        features: ['text'],
        num_rows: 4358
    })
    train: Dataset({
        features: ['text'],
        num_rows: 1801350
    })
    validation: Dataset({
        features: ['text'],
        num_rows: 3760
    })
})

In [5]:
#sélectionne aléatoirement n ligne de texte et les nettois XXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXXX pas encore bien nettoyer
n = 10000
text_data = raw_ds['train'].shuffle()[:n]['text']
cleaned_text_data = []

for line in text_data:

    line = line.replace('@-@', '-')
    line = line.replace(' @,@ ', ',')
    line = line.replace(' @.@ ', '.')
    line = re.sub(r'\s+', ' ', line).strip()
    line = line.replace("\\'", "'") # ne marche pas je veux remplacer les \' par ' mais j'y arrive pas
    
    # 3. Avoid adding empty lines
    if line:  # Only add non-empty lines
        cleaned_text_data.append(line)
cleaned_text_data = [ line for line in cleaned_text_data 
                        if not (line.startswith('=') and line.endswith('='))
]



On as pris au hasard tant de ligne de texte:

In [6]:
len(cleaned_text_data)

4767

Bien qu'on ait les cpu implémenter, on ne peut pas faire tourner les cpu avec le model gpt2 xl car on as des problème de mémoire (même lorsqu'on ne mets qu'une ligne) donc on calcule C par batch comme dans la méthode décrite précédemment via le CPU

In [10]:
test = Calcul_C(cleaned_text_data[:100])
test.nb_ks

11839

In [7]:
cleaned_text_data = cleaned_text_data[:1000]

In [8]:
N = 0
i=0
nb_ks = []
Cs = []
file_name = 'C_medium.pt'
model_name = 'gpt2-medium'
while N <= 50000:
    t1 = time.time()
    instance = Calcul_C(prompts=cleaned_text_data[i*5:i*5+5], model_name= model_name)
    nbk = instance.nb_ks
    nb_ks.append(nbk)
    instance.run()
    Cs.append(instance.C)
    N+=nbk
    i+=1
    t2 = time.time()
    elapsed = t2 - t1
    minutes = int(elapsed // 60)
    seconds = int(elapsed % 60)
    print(f"étape {i} ; nombre de k dans l'étape : {nbk};  N = {N}; Execution time: {minutes} minute(s) and {seconds} second(s)")




étape 1 ; nombre de k dans l'étape : 535;  N = 535; Execution time: 0 minute(s) and 22 second(s)
étape 2 ; nombre de k dans l'étape : 405;  N = 940; Execution time: 0 minute(s) and 5 second(s)
étape 3 ; nombre de k dans l'étape : 525;  N = 1465; Execution time: 0 minute(s) and 6 second(s)
étape 4 ; nombre de k dans l'étape : 719;  N = 2184; Execution time: 0 minute(s) and 7 second(s)
étape 5 ; nombre de k dans l'étape : 351;  N = 2535; Execution time: 0 minute(s) and 3 second(s)
étape 6 ; nombre de k dans l'étape : 481;  N = 3016; Execution time: 0 minute(s) and 4 second(s)
étape 7 ; nombre de k dans l'étape : 723;  N = 3739; Execution time: 0 minute(s) and 5 second(s)
étape 8 ; nombre de k dans l'étape : 945;  N = 4684; Execution time: 0 minute(s) and 8 second(s)
étape 9 ; nombre de k dans l'étape : 364;  N = 5048; Execution time: 0 minute(s) and 4 second(s)
étape 10 ; nombre de k dans l'étape : 464;  N = 5512; Execution time: 0 minute(s) and 6 second(s)
étape 11 ; nombre de k dans l'

In [10]:
weighted_sum = sum(C * nbk for C, nbk in zip(Cs, nb_ks))
C_gpt_xl = weighted_sum / N

On enregistre ce C qui utilise 50466 ks puis on va calculer un autre C sur envire 50000 ks pour enfin calculer un C qui utilise 100000 ks

In [11]:
metadata = {
    'model_name': model_name,
    'number_of_ks': N}
torch.save({'C': C_gpt_xl, 'Metadata': metadata }, file_name)



In [14]:
N = 0
nb_ks = []
Cs = []
while N <= 50000:
    t1 = time.time()
    instance = Calcul_C(prompts=cleaned_text_data[i*5:i*5+5], model_name= model_name, device='cpu')
    nbk = instance.nb_ks
    nb_ks.append(nbk)
    instance.run()
    Cs.append(instance.C)
    N+=nbk
    i+=1
    t2 = time.time()
    elapsed = t2 - t1
    minutes = int(elapsed // 60)
    seconds = int(elapsed % 60)
    print(f"étape {i} ; nombre de k dans l'étape : {nbk};  N = {N}; Execution time: {minutes} minute(s) and {seconds} second(s)")

étape 88 ; nombre de k dans l'étape : 762;  N = 762; Execution time: 0 minute(s) and 7 second(s)
étape 89 ; nombre de k dans l'étape : 749;  N = 1511; Execution time: 0 minute(s) and 7 second(s)
étape 90 ; nombre de k dans l'étape : 671;  N = 2182; Execution time: 0 minute(s) and 6 second(s)
étape 91 ; nombre de k dans l'étape : 623;  N = 2805; Execution time: 0 minute(s) and 5 second(s)
étape 92 ; nombre de k dans l'étape : 685;  N = 3490; Execution time: 0 minute(s) and 6 second(s)
étape 93 ; nombre de k dans l'étape : 881;  N = 4371; Execution time: 0 minute(s) and 7 second(s)
étape 94 ; nombre de k dans l'étape : 519;  N = 4890; Execution time: 0 minute(s) and 4 second(s)
étape 95 ; nombre de k dans l'étape : 545;  N = 5435; Execution time: 0 minute(s) and 4 second(s)
étape 96 ; nombre de k dans l'étape : 427;  N = 5862; Execution time: 0 minute(s) and 4 second(s)
étape 97 ; nombre de k dans l'étape : 586;  N = 6448; Execution time: 0 minute(s) and 5 second(s)
étape 98 ; nombre de 

In [15]:
weighted_sum = sum(C * nbk for C, nbk in zip(Cs, nb_ks))
C_gpt_xl = weighted_sum / N

In [16]:
metadata2 = {
    'model_name': model_name,
    'number_of_ks': N
}
checkpoint = torch.load(file_name)
checkpoint['C2'] = C_gpt_xl
checkpoint['Metadata2'] = metadata2
torch.save(checkpoint, file_name) 

In [17]:
checkpoint = torch.load(file_name)
C100 = (checkpoint['Metadata']['number_of_ks']*checkpoint['C']+checkpoint['Metadata2']['number_of_ks']*checkpoint['C2'])/(checkpoint['Metadata']['number_of_ks']+checkpoint['Metadata2']['number_of_ks'])
dico1 = {'C':C100, 'model_name': model_name, 'number_of_ks': checkpoint['Metadata']['number_of_ks']+checkpoint['Metadata2']['number_of_ks'] }
dico2 = {'C':checkpoint['C'], 'model_name': model_name, 'number_of_ks': checkpoint['Metadata']['number_of_ks'] }
dico3 = {'C':checkpoint['C2'], 'model_name': model_name, 'number_of_ks': checkpoint['Metadata2']['number_of_ks'] }
data = {1 : dico1, 2: dico2 , 3 : dico3}

In [18]:
torch.save(data, file_name) 

On as ainsi calculé la matrice C. On peut accéder à C comme suit.

In [22]:
data = torch.load('C.pt')
data

{1: {'C': tensor([[0.0299, 0.0021, 0.0013,  ..., 0.0018, 0.0012, 0.0019],
          [0.0021, 0.0178, 0.0044,  ..., 0.0039, 0.0018, 0.0044],
          [0.0013, 0.0044, 0.0311,  ..., 0.0040, 0.0023, 0.0043],
          ...,
          [0.0018, 0.0039, 0.0040,  ..., 0.0302, 0.0033, 0.0042],
          [0.0012, 0.0018, 0.0023,  ..., 0.0033, 0.0400, 0.0019],
          [0.0019, 0.0044, 0.0043,  ..., 0.0042, 0.0019, 0.0137]]),
  'model_name': 'gpt2-xl',
  'number_of_ks': 100254},
 2: {'C': tensor([[0.0292, 0.0023, 0.0014,  ..., 0.0017, 0.0011, 0.0020],
          [0.0023, 0.0184, 0.0043,  ..., 0.0038, 0.0018, 0.0044],
          [0.0014, 0.0043, 0.0303,  ..., 0.0039, 0.0023, 0.0044],
          ...,
          [0.0017, 0.0038, 0.0039,  ..., 0.0319, 0.0035, 0.0042],
          [0.0011, 0.0018, 0.0023,  ..., 0.0035, 0.0372, 0.0021],
          [0.0020, 0.0044, 0.0044,  ..., 0.0042, 0.0021, 0.0133]]),
  'model_name': 'gpt2-xl',
  'number_of_ks': 50170},
 3: {'C': tensor([[0.0306, 0.0020, 0.0011,  ..., 0.

In [25]:
data[1]['C']

tensor([[0.0299, 0.0021, 0.0013,  ..., 0.0018, 0.0012, 0.0019],
        [0.0021, 0.0178, 0.0044,  ..., 0.0039, 0.0018, 0.0044],
        [0.0013, 0.0044, 0.0311,  ..., 0.0040, 0.0023, 0.0043],
        ...,
        [0.0018, 0.0039, 0.0040,  ..., 0.0302, 0.0033, 0.0042],
        [0.0012, 0.0018, 0.0023,  ..., 0.0033, 0.0400, 0.0019],
        [0.0019, 0.0044, 0.0043,  ..., 0.0042, 0.0019, 0.0137]])